In [1]:
import sys
from pathlib import Path
from os import getcwd

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['C:\\sesa\\dev\\notebooks', 'C:\\ProgramData\\Anaconda3\\python38.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', '', 'C:\\Users\\terc.vinicius\\AppData\\Roaming\\Python\\Python38\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\terc.vinicius\\.ipython', 'C:\\sesa\\dev']


In [2]:
from os.path import join
import pandas as pd
import numpy as np
from sys import exit
from datetime import time

pd.set_option('display.max_columns', None)

In [3]:
from bulletin.systems.notifica import Notifica
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.utils.normalize import normalize_hash, normalize_labels


---------------------------------------------------------------

root: C:\sesa\dev\bulletin
parent: C:\sesa\dev
default_input: C:\sesa\dev\input
default_output: C:\sesa\dev\output

---------------------------------------------------------------



In [4]:
notifica = Notifica()
notifica.load()
print(f"notifica len: {len(notifica)}")
# notifica.hashes()
# notifica.save(replace=True)
casosn = notifica.df
print('duplicados ',len(casosn.loc[casosn.duplicated('hash_resid',keep='first')]))
casosn = casosn.drop_duplicates('hash_resid',keep='first')
print(f"casosn len: {len(casosn)}")

['comorbidades', 'instituicao', 'notifica', 'update_notifica']
loading Notifica from pkl time elepsed:00:00:04
notifica len: 1200546
duplicados  12626
casosn len: 1187920


In [5]:
cc = CasosConfirmados()
cc.load()
print(f"cc len: {len(cc)}")
# cc.hashes()
# cc.save(replace=True)
casosc = cc.df

['cc_19_06_2021']
loading Casos Confirmados from pkl time elepsed:00:00:03
cc len: 1192936


In [6]:
casosn_id = casosn[['id','hash_resid']].set_index('hash_resid').rename(columns={'id':'id_notifica'})
casosc = casosc.set_index('hash_resid')
casosc.update(casosn_id)
casosc.reset_index(inplace=True)
casosc['id_notifica'] = casosc['id_notifica'].astype(int) 
print(len(casosc.loc[casosc['id_notifica']==-1]))
casosc.loc[casosc['id_notifica']==-1,['id_notifica','data_comunicacao']].groupby('data_comunicacao').count()

87488


,id_notifica
data_comunicacao,
2020-03-12,6
2020-03-17,4
2020-03-18,4
2020-03-19,7
2020-03-20,12
...,...
2021-06-14,83
2021-06-15,160
2021-06-16,21


In [7]:
casosn = casosn.drop(casosn.loc[casosn['id'].isin(casosc['id_notifica'])].index)
len(casosn)

82472

In [8]:
cc.df = casosc
cc.save(replace=True)

saving Casos Confirmados to pkl time elepsed:00:00:10
